In [1]:
%load_ext autoreload

import numpy as np
from PIL import Image, ImageDraw

import tensorflow as tf
tf.enable_eager_execution()

import torch

In [10]:
np.random.randn(1000, 2).shape

(1000, 2)

In [7]:
from collections import namedtuple

import torch.nn.functional as F
from torchvision import datasets, models, transforms

m = models.vgg16_bn(pretrained=True).features
print(m)


transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])
x = transform(Image.open("/workspace/CoordConv-pytorch/data/pin/0189dc019d927ba127ac8528707d445e.jpg"))
x = x.unsqueeze(0)

# y = vgg(x)
y = m(x)
print(y.shape)
y = F.avg_pool2d(y, 3, padding=1)
y.shape

# y.detach().numpy()


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU(inplace=True)
  (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU(inplace=True)
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

torch.Size([1, 512, 5, 3])

In [54]:
def pairwise_distance(data1, data2=None, device=-1):
	r'''
	using broadcast mechanism to calculate pairwise ecludian distance of data
	the input data is N*M matrix, where M is the dimension
	we first expand the N*M matrix into N*1*M matrix A and 1*N*M matrix B
	then a simple elementwise operation of A and B will handle the pairwise operation of points represented by data
	'''
	if data2 is None:
		data2 = data1 

	if device!=-1:
		data1, data2 = data1.cuda(device), data2.cuda(device)

	#N*1*M
	A = data1.unsqueeze(dim=1)

	#1*N*M
	B = data2.unsqueeze(dim=0)

	dis = (A-B)**2.0
	#return N*N matrix for pairwise distance
	dis = dis.sum(dim=-1).squeeze()
	return dis


def forgy(X, n_clusters):
	_len = len(X)
	indices = np.random.choice(_len, n_clusters)
	initial_state = X[indices]
	return initial_state


def lloyd(X, n_clusters, device=0, tol=1e-4):
	X = torch.from_numpy(X).float()
#     .cuda(device)

	initial_state = forgy(X, n_clusters)


	while True:
		dis = pairwise_distance(X, initial_state)

		choice_cluster = torch.argmin(dis, dim=1)

		initial_state_pre = initial_state.clone()

		for index in range(n_clusters):
			selected = torch.nonzero(choice_cluster==index).squeeze()

			selected = torch.index_select(X, 0, selected)
			initial_state[index] = selected.mean(dim=0)
		

		center_shift = torch.sum(torch.sqrt(torch.sum((initial_state - initial_state_pre) ** 2, dim=1)))

		if center_shift ** 2 < tol:
			break

	return choice_cluster.cpu().numpy(), initial_state.cpu().numpy()

A = np.concatenate([np.random.randn(100, 200), np.random.randn(100, 200)+3, np.random.randn(100, 200)+6], axis=0)

#lloyd(X, n_clusters, device=0, tol=1e-4)
clusters_index, centers = lloyd(A, 3, device=-1, tol=1e-4)
clusters_index
centers

array([[6.144625 , 5.944611 , 6.1140833, 6.1475325, 5.9054418, 5.822255 ,
        5.593646 , 6.059982 , 6.483781 , 5.889703 , 6.2198386, 5.6442757,
        5.4158716, 6.0971427, 6.301312 , 5.8527713, 6.0597453, 5.635212 ,
        5.7684083, 6.2265472, 6.2389145, 5.859596 , 6.095941 , 5.935408 ,
        6.0333223, 6.215909 , 5.8792686, 5.91635  , 5.9047155, 6.205185 ,
        5.846923 , 6.233918 , 5.5446906, 5.8319316, 5.9259877, 5.958965 ,
        5.628144 , 5.866509 , 5.9294066, 6.2537155, 5.896252 , 5.4413447,
        5.939777 , 6.3528495, 5.5589347, 5.679134 , 5.9253144, 5.956631 ,
        5.4057198, 5.689386 , 6.215873 , 6.3554225, 5.7686996, 5.8904533,
        6.1087456, 6.1338105, 5.7976418, 6.0996213, 6.068554 , 5.8629956,
        5.54     , 5.9927263, 6.0861096, 5.435902 , 5.8434224, 6.0457845,
        6.111389 , 5.9799685, 5.864781 , 6.088475 , 5.8947473, 6.010778 ,
        5.9563384, 5.7499785, 6.049435 , 5.5257716, 6.036753 , 5.9806967,
        6.473262 , 5.7111087, 6.148232

In [44]:
# y[0].shape
import torch.nn.functional as F
y.shape
# y = F.avg_pool2d(y, (3, 3), padding=1)
# y.view(1, -1).shape

torch.Size([1, 512, 2, 1])

In [2]:
# !pip install -q -U tensorflow_hub
import tensorflow_hub as hub

def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img


content_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
content_image = load_img(content_path)
x = tf.keras.applications.vgg19.preprocess_input(content_image*255)
x = tf.image.resize(x, (224, 224))

feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" #@param {type:"string"}
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(224,224,3))
feature_batch = feature_extractor_layer(x)

print(feature_batch)


tf.Tensor([[1.5332396  0.         1.1868494  ... 0.         0.06440655 0.        ]], shape=(1, 1280), dtype=float32)


In [ ]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)


def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img


content_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
# https://commons.wikimedia.org/wiki/File:Vassily_Kandinsky,_1913_-_Composition_7.jpg
# style_path = tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

content_image = load_img(content_path)

x = tf.keras.applications.vgg19.preprocess_input(content_image*255)
x = tf.image.resize(x, (224, 224))
# vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')
# prediction_probabilities = vgg(x)
# prediction_probabilities.shape

# predicted_top_5 = tf.keras.applications.vgg19.decode_predictions(prediction_probabilities.numpy())[0]
# [(class_name, prob) for (number, class_name, prob) in predicted_top_5]

vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')


In [ ]:
y = vgg(x)
# y = tf.keras.layers.Flatten(y)
y.shape

# for layer in vgg.layers:
#   print(layer.name)


In [ ]:
def _reward(xy_a: np.array, xy_b: np.array):
    dist = np.linalg.norm(xy_a - xy_b, axis=1)
    r = -1 * dist / 2 + 1
    r = np.clip(r, -1, None)
    r = np.sum(r)
#     elif r > 0:
#         r *= 0.05 ** self.cur_step  # 衰退因子
    return r

a, b = np.random.rand(2,4), np.random.rand(2,4)
print(_reward(a, b))
# _reward(a[0], b[0]) + _reward(a[1], b[1])

In [ ]:
%autoreload 2
%cd /workspace/LearningToPaint/rllib

from canvas_env import CanvasEnv

e = CanvasEnv({})
obs = e.reset()

def to_image(array):
    im = (array * 255).astype(np.uint8)
    im = np.squeeze(im, 2)
    return Image.fromarray(im)

acc = 0
for _ in range(5):
    obs, reward, done, _ = e.step([3, 3])
    acc += reward
    if done:
        break
acc

# to_image(obs['target_im'])
# to_image(obs['cur_im'])
# im = obs['cur_im']
# obs['obj_status']

# _obs['target_im'] == obs['target_im']

In [ ]:
a, (b, c) = [1, np.array([2,3])]
c

In [ ]:
from PIL import Image, ImageDraw

im = Image.new("L", (7, 10))
draw = ImageDraw.Draw(im)
draw.rectangle([2, 2, 4, 4], fill=255)

# np.array(im).shape
# tf.expand_dims(tf.image.convert_image_dtype(np.array(im), tf.float32), 2)
# x = np.array([[1]])
# x[0, 0]

# tf.keras.preprocessing.image.img_to_array(im).shape
# np.array(im)
x = np.array(im)
x = np.expand_dims(x, -1)
x = np.expand_dims(x, 0)
x.shape

In [ ]:
# a = tf.random.normal((1,2,3,4))
a = np.array([1,2,3,4])
# x.shape == self.shape and np.all(x >= self.low) and np.all(x <= self.high)
# def at():
#     print('at')
# def af():
#     print('af')
# # def a: print('a')
# tf.cond(np.all(a >= 1), af, af )

np.all(a>=1)

In [ ]:
x = np.array([1,2,3])
x * 2

In [ ]:
# from gym import spaces
# from ray.rllib.models.catalog import ModelCatalog

# # DICT_SPACE = spaces.Tuple([
# #     spaces.Discrete(100),
# #     spaces.Box(low=-100, high=100, shape=(3, ))
# # ])

# a = spaces.Box(low=-100, high=100, shape=(3,))
# _dist_class, logit_dim = ModelCatalog.get_action_dist(a, None)
# print(_dist_class, logit_dim)

# tf.random.categorical(tf.math.log([[10., 10.]]), 5)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Activation
from tensorflow.keras.models import Model

num_outputs = 1 + 2*2 # logits for num_objects (one each) & (mu, sdd) for coordinates (x0, y0) each

# define a cnn model
input_img = Input(shape=(9, 9, 1))  # (H, W, C)
x = Conv2D(8, 3, activation="relu", padding="same")(input_img)
x = Conv2D(32, 3, activation="relu", padding="same")(x)
x = Conv2D(64, 3, activation="relu", padding="same")(x)
x = Conv2D(64, 3, padding="same")(x)
x = Conv2D(1, 3, padding="same")(x)
out = Flatten()(x)

cnn_model= Model(input_img, out)
cnn_model.summary()

cur_im = Input(shape=(9, 9, 1))
target_im = Input(shape=(9, 9, 1))

# The vision model will be shared, weights and all
out_cur = cnn_model(cur_im)
out_target = cnn_model(target_im)
obj_status = Input(shape=(1, 4))  # (num_obj, 4=(x0, y0, x1, y1))

x = Flatten()(obj_status)
x = tf.keras.layers.concatenate([out_cur, out_target, x])
x = Dense(128, activation="relu")(x)
x = Dense(128, activation="relu")(x)
out = Dense(num_outputs, activation=None)(x)

model = Model([cur_im, target_im, obj_status], out)

model([tf.random.normal((10, 9, 9, 1)), tf.random.normal((10, 9, 9, 1)), tf.random.normal((10, 1, 4))])

In [ ]:
import torch
from torch.distributions.normal import Normal
from torch.distributions.categorical import Categorical

# m = Categorical(torch.tensor([0.0]), torch.tensor([1.0]))
m = Categorical(torch.tensor([ 0.1, 0.25, 0.25, 0.25 ]))
a = m.sample()
print(a)
m.log_prob(a)

# from gym import error, spaces

# x = spaces.Tuple(
#             (spaces.Discrete(1), spaces.Box(low=0, high=1, shape=(1,)))
#         )
# for s in x:
#     print(s)


# a = torch.rand(1, 3, 3)
# b = torch.rand(1, 3, 3)
# c = torch.rand(1, 4)
# print(a, b, c)

# z = torch.cat((a.view(-1), b.view(-1), c.view(-1)), dim=0)
# a, b = torch.split(z, 2*3*3, dim=0)
# print(a.view(2,3,3), b.view(1,4))


In [ ]:
FixedNormal = torch.distributions.Normal

def init(module, weight_init, bias_init, gain=1):
    weight_init(module.weight.data, gain=gain)
    bias_init(module.bias.data)
    return module

class AddBias(nn.Module):
    def __init__(self, bias):
        super(AddBias, self).__init__()
        self._bias = nn.Parameter(bias.unsqueeze(1))

    def forward(self, x):
        if x.dim() == 2:
            bias = self._bias.t().view(1, -1)
        else:
            bias = self._bias.t().view(1, -1, 1, 1)

        return x + bias

class DiagGaussian(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(DiagGaussian, self).__init__()

        init_ = lambda m: init(m, nn.init.orthogonal_, lambda x: nn.init.
                               constant_(x, 0))

        self.fc_mean = init_(nn.Linear(num_inputs, num_outputs))
        self.logstd = AddBias(torch.zeros(num_outputs))

    def forward(self, x):
        action_mean = self.fc_mean(x)

        #  An ugly hack for my KFAC implementation.
        zeros = torch.zeros(action_mean.size())
        if x.is_cuda:
            zeros = zeros.cuda()

        action_logstd = self.logstd(zeros)
        print(action_mean, action_logstd.exp())
        return FixedNormal(action_mean, action_logstd.exp())

m = DiagGaussian(512, 2)
x = torch.rand((1, 512))
dist = m(x)
dist.sample()[0]

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


class NNBase(nn.Module):
    def __init__(self, recurrent, recurrent_input_size, hidden_size):
        super(NNBase, self).__init__()

        self._hidden_size = hidden_size
        self._recurrent = recurrent

        if recurrent:
            self.gru = nn.GRU(recurrent_input_size, hidden_size)
            for name, param in self.gru.named_parameters():
                if 'bias' in name:
                    nn.init.constant_(param, 0)
                elif 'weight' in name:
                    nn.init.orthogonal_(param)

    @property
    def is_recurrent(self):
        return self._recurrent

    @property
    def recurrent_hidden_state_size(self):
        if self._recurrent:
            return self._hidden_size
        return 1

    @property
    def output_size(self):
        return self._hidden_size

    def _forward_gru(self, x, hxs, masks):
        if x.size(0) == hxs.size(0):
            x, hxs = self.gru(x.unsqueeze(0), (hxs * masks).unsqueeze(0))
            x = x.squeeze(0)
            hxs = hxs.squeeze(0)
        else:
            # x is a (T, N, -1) tensor that has been flatten to (T * N, -1)
            N = hxs.size(0)
            T = int(x.size(0) / N)

            # unflatten
            x = x.view(T, N, x.size(1))

            # Same deal with masks
            masks = masks.view(T, N)

            # Let's figure out which steps in the sequence have a zero for any agent
            # We will always assume t=0 has a zero in it as that makes the logic cleaner
            has_zeros = ((masks[1:] == 0.0) \
                            .any(dim=-1)
                            .nonzero()
                            .squeeze()
                            .cpu())

            # +1 to correct the masks[1:]
            if has_zeros.dim() == 0:
                # Deal with scalar
                has_zeros = [has_zeros.item() + 1]
            else:
                has_zeros = (has_zeros + 1).numpy().tolist()

            # add t=0 and t=T to the list
            has_zeros = [0] + has_zeros + [T]

            hxs = hxs.unsqueeze(0)
            outputs = []
            for i in range(len(has_zeros) - 1):
                # We can now process steps that don't have any zeros in masks together!
                # This is much faster
                start_idx = has_zeros[i]
                end_idx = has_zeros[i + 1]

                rnn_scores, hxs = self.gru(
                    x[start_idx:end_idx],
                    hxs * masks[start_idx].view(1, -1, 1))

                outputs.append(rnn_scores)

            # assert len(outputs) == T
            # x is a (T, N, -1) tensor
            x = torch.cat(outputs, dim=0)
            # flatten
            x = x.view(T * N, -1)
            hxs = hxs.squeeze(0)

        return x, hxs

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

    
class MyCNNBase(NNBase):
    def __init__(self, num_inputs, recurrent=False, hidden_size=512):
        super(MyCNNBase, self).__init__(recurrent, hidden_size, hidden_size)
        
        init_ = lambda m: init(m, nn.init.orthogonal_, lambda x: nn.init.
                               constant_(x, 0), nn.init.calculate_gain('relu'))

        self.main = nn.Sequential(
            init_(nn.Conv2d(num_inputs, 8, 3, padding=1)), nn.ReLU(),
            init_(nn.Conv2d(8, 16, 3, padding=1)), nn.ReLU(),
            init_(nn.Conv2d(16, 16, 3, padding=1)), nn.ReLU(), Flatten(),
            init_(nn.Linear(16 * 64 * 64, hidden_size)), nn.ReLU()
        )

        init_ = lambda m: init(m, nn.init.orthogonal_, lambda x: nn.init.
                               constant_(x, 0))

        self.critic_linear = init_(nn.Linear(hidden_size, 1))

        self.train()

    def forward(self, inputs, rnn_hxs=None, masks=None):
        x = self.main(inputs)
        print(x.shape)

        if self.is_recurrent:
            x, rnn_hxs = self._forward_gru(x, rnn_hxs, masks)

        return self.critic_linear(x), x, rnn_hxs
    
model = MyCNNBase(2)
x = torch.rand((1,2,64,64))
model(x)

In [ ]:
# %cd /notebooks/LearningToPaint/gym-canvas
# !pip uninstall -e .
# !python setup.py develop -u


import gym
from baselines.common.vec_env.util import dict_to_obs, obs_space_info, obs_to_dict

import sys
sys.path.append("/notebooks/LearningToPaint/gym-canvas")
import gym_canvas

env = gym.make('gym_canvas:canvas-v0')
obs_space_info(env.observation_space)


In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import torch
import torchvision.transforms as transforms


transform = transforms.ToTensor()

width = 8

im = Image.new("L", (width, width))
draw = ImageDraw.Draw(im)
draw.rectangle([2, 2, 4, 4], fill=255)
im1 = transform(im)

im = Image.new("L", (width, width))
draw = ImageDraw.Draw(im)
draw.rectangle([2, 2, 4, 4], fill=255)
im2 = transform(im)


# x, y = np.random.randint(2, size=2)
# x
# type(y)

torch.cat((im1, im2), 0).shape

In [ ]:
%matplotlib inline

import numpy as np
import cv2
import scipy.misc
from PIL import Image
import matplotlib.pyplot as plt

from Renderer.stroke_gen import draw

f = np.random.uniform(0, 1, 10)
print(f)
im = draw(f, 300)

plt.imshow(im, cmap='gray')

In [ ]:
from PIL import Image, ImageDraw
from Renderer.stroke_gen import draw_rect

# width = 3
# im = Image.new('F', (width, width))

# draw = ImageDraw.Draw(im)
# # draw.line((0, 0) + im.size, fill=128)
# # draw.line((0, im.size[1], im.size[0], 0), fill=128)
# draw.rectangle([1, 1, 2, 2], fill=1)
# # plt.imshow(np.array(im), cmap='gray')
# plt.imshow(im, vmin=0, vmax=1)

im = draw_rect([0.5, 0.5, 1, 1], width=4)
# plt.imshow(im, vmin=0, vmax=1)
plt.imshow(1-im, cmap='gray')
print(im)

In [ ]:
def rand_draw(draw_fn=draw_rect, n_strokes=3, width=128):
    canvas = np.zeros((width, width, 3), dtype=int)
    for i in range(n_strokes):
        x = np.random.rand(4)
        stroke = draw_fn(x, width)
        stroke = np.expand_dims(stroke, axis=2)
        color = np.random.randint(255, size=(3))
        canvas = canvas * (1 - stroke) + stroke * color
    return canvas.astype(int)

plt.imshow(rand_draw(width=128))
# plt.imshow(1-im, cmap='gray')

In [ ]:
import torch
from Renderer.model import FCN

action_dim = 4

stroke_net = FCN(num_input=action_dim)
stroke_net.load_state_dict(torch.load("../renderer.pkl"))

# x = torch.rand(1, action_dim)

print(draw_rect([0, 0, 0.5, 0.5]))
x = torch.from_numpy(np.array([[0, 0, 0.5, 0.5]], dtype=np.float32))
print(x.shape)
# x = np.random.rand(action_dim)
stroke_net(x)

In [ ]:
from env import Paint

env = Paint(1, 3)
env.load_data()
obs = env.reset()
obs = env.reset_with_gen()

# print(obs[0, 3:6])
im = np.array(obs[0, 3:6])
plt.imshow(np.transpose(im, (1, 2, 0)))


In [ ]:
import torch
from Renderer.model import FCN
from Renderer.stroke_gen import draw_rect
# from DRL.ddpg import decode

Decoder = FCN(num_input=action_dim)
Decoder.load_state_dict(torch.load("../renderer.pkl"))

print(draw_rect([0, 0, 1, 1]).shape)

def decode(x, canvas, width=128):
    # x: (B * 5) * (10 + 3), 5: 5 steps, 10+3: action output
    x = x.view(-1, action_dim + 3)
#     stroke = 1 - Decoder(x[:, :action_dim])
    
    # debug: use draw_fn instead
    stroke = []
    for i in range(batch):
        stroke.append(draw_rect(x[i, :action_dim]))
    stroke = torch.tensor(stroke)

    stroke = stroke.view(-1, width, width, 1)  # shape: (b, w, w, 1)
    print(stroke.shape)
#     color_stroke = stroke * x[:, -3:].view(-1, 1, 1, 3)
    color_stroke = stroke * x[:, -3:].view(-1, 1, 1, 3)  # shape: (b, w, w, 3)
#     print(color_stroke.shape)
    stroke = stroke.permute(0, 3, 1, 2).view(-1, n_frames_per_step, 1, width, width)
    color_stroke = color_stroke.permute(0, 3, 1, 2).view(-1, n_frames_per_step, 3, width, width)
    for i in range(n_frames_per_step):
        canvas = canvas * (1 - stroke[:, i]) + color_stroke[:, i]
    return canvas

n_frames_per_step = 1
batch = 1
width = 128

canvas = torch.zeros(batch,3,width,width)# [B x 3 x width x width]

_action = torch.rand(batch, 4)
color = torch.rand(batch, 3)
action = torch.cat((_action, color), 1)

canvas_out =  decode(action, canvas, width)

im = canvas_out.squeeze(0).detach().numpy()
im = np.transpose(im, (1, 2, 0))
# canvas = decode(action, canvas)
plt.imshow(im)

In [ ]:
from env import Paint
from DRL.ddpg import DDPG
from utils.util import to_numpy
from utils.tensorboard import TensorBoard

exp = os.path.abspath(".").split("/")[-1]
writer = TensorBoard("../train_log/{}".format(exp))

agent = DDPG(1, 1, 5, 0.001, 0.95 ** 5, 800, writer, None, "./model")

env = Paint(1, 5)
obs = env.reset_with_gen()
agent.reset(obs, 0)
action = agent.select_action(obs, noise_factor=0)
obs, reward, done, _ = env.step(torch.tensor(action))
# env.step()

# print(obs.shape)
# env.gt[0].cpu().data.numpy()
cv = to_numpy(env.canvas[0])



In [ ]:
# x = [np.random.rand(4) for _ in range(3)]
# x = np.stack(x)
# print(x)
# # x.shape

# a = np.random.rand(3)
# print(a)
# b = np.random.rand(4)
# np.concatenate((a,b))
x = torch.tensor([1,2,3,4])
x.view(2, -1)
